In [0]:
from functools import reduce
from typing import Dict, List, Any
from datetime import datetime
from datetime import timedelta 
import pkg_resources
import pyspark.sql
import os, glob
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, MapType, StructField, StructType, FloatType
import pyspark.sql.types as T
import importlib
import logging
import pkgutil
import json

In [0]:
%run Shared/functions/Functions_delta

In [0]:
input_file =dbutils.widgets.get("input_file")

#get input parameters
import json
with open(input_file, 'r') as file:
  data = file.read()
jsonObject = json.loads(data)

inputs: Dict[str, Dict[str, Any]] = jsonObject["jointransform"]
query= inputs["query"]
business_key= inputs["bussiness_keys"]
sort_key= inputs["sort_key"]   #26 Nov: Sort_key is mandatory for avoiding duplicates
primary_key=inputs["primary_key"]
non_align_columns=inputs["non_align_columns"]
 
loadstrategy = inputs["loadstrategy"]
inputTable=inputs["inputTable"]
outputTable=inputs["outputTable"]
outputExtract=inputs["outputExtract"]

adlsPath=inputs["adlsPath"]
adlsfolder=inputs["adlsfolder"]
hour_partition=inputs["hour_partition"]
date_column= "current_date" if not inputs["date_column"].strip() else inputs["date_column"].strip()

trans_name= "jointransform"

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1390557124504346> in <module> 
 7 jsonObject = json . loads ( data ) 
 8 
 ----> 9 inputs : Dict [ str , Dict [ str , Any ] ] = jsonObject [ "jointransform" ] 
 10 query = inputs [ "query" ] 
 11 business_key = inputs [ "bussiness_keys" ] 

 NameError : name 'Dict' is not defined

In [0]:
sources: Dict[str, Dict[str, Any]] =inputs["sources"]
if len(sources)!=0:
  input_s: Dict[str, pyspark.sql.DataFrame] = extract(sources)
  for key in input_s:
     input_s[key].registerTempTable(key)

In [0]:
#updated stroed proc logic for date loop: provide the start date and end date in jobinput.json file and python datetime format of your date column
#note: 1. query should not have any entire null column, spark won't accept it.
#######2. default values for both parameter are current date in '%Y-%m-%d' default format.
try:
  start_dt = inputs["start_dt"]["value"]
  if start_dt == '':
      start_dt = str(datetime.now().date() + timedelta(days=-1))
except:
  start_dt = str(datetime.now().date() + timedelta(days=-1)).date()
  
try:
  end_dt = inputs["end_dt"]["value"]
  if end_dt == '':
    end_dt = str(datetime.now().date())
except:
  end_dt = str(datetime.now().date())
  
try:
  start_dt_format = inputs["start_dt"]["python_format"]
  if start_dt_format == '':
    start_dt_format = '%Y-%m-%d'
except:
  start_dt_format = '%Y-%m-%d'
  
try:
  end_dt_format = inputs["end_dt"]["python_format"]
  if start_dt_format == '':
    end_dt_format = '%Y-%m-%d'
except:
  end_dt_format = '%Y-%m-%d'

ReportDate = datetime.strptime(start_dt, start_dt_format)
end_dt= datetime.strptime(end_dt, end_dt_format)

while ReportDate <= end_dt:
  for step in inputs['SP']:
    temptable = inputs['SP'][step]["temptable"] #name for temporary table from input file->SP->step1->temp
    stepdf = spark.sql(inputs['SP'][step]["query"].format(ReportDate))
    stepdf.registerTempTable(temptable)

  final_table = spark.sql("select * from " + temptable)
  if final_table.count != 0:
    final_table.write.mode("append").saveAsTable("final_table_temp")
  ReportDate = ReportDate + timedelta(days=1)


In [0]:
%run ./transform_delta

Out[40]: '\nDec 01, 2020: When sqlrefresh true then newly added \'sqlrefresh_query\' parameter can refresh only 7 days prior data from SQL server. \nNote: If sqlrefresh_query = "" or sqlrefresh_query is not provided in inputjson then entire SQL table will be refreshed into ADLS.\n\nDec02, 2020: when sqlrefresh true/false save SQL table in temporary ADLS table.\noverwrite will directly overwrite ADLS and SQL data\nAppend: SQL refresh false: Incoming data will append to ADLS and SQL data\nAppend: SQL refresh true: SQL data will merge into ADLS and then incoming data will merge into ADLS and SQL\nupsert: if sqlrefresh is true or false then SQL table data will be loaded into temp table.\n sqlrefresh: false: incoming data will merge into ADLS and then merge into internal temp(sql refresh data) with name outputDB.outputTable+"_temp". Temp table extract will load into sql\n sqlrefresh: true: incoming data will merge into ADLS and then merge into temp(sql refresh data). Temp table extract will load into sql. After this Temp table extract will merge into ADLS table\n - Temp table will be dropped once load completed.\nthen merge final sql table data into ADLS\n#Dec, 12, 2020: 1. sqlSchema parameter indicates schema name of the table. if not provided or blank then default "dbo" schema will be used. \n 2. truncate parameter is added for truncate in sql table write. It is added incase of overwrite and if truncate option should be \'False\'.\n'

['year', 'month', 'date']

In [0]:
%sql
drop table final_table_temp